In [1]:
using FCA, Combinatorics, LinearAlgebra, Statistics

┌ Info: Recompiling stale cache file C:\Users\Hao Wu\.julia\compiled\v1.0\FCA\bABXD.ji for FCA [7d5165da-9cd4-5962-a300-0a643be0618e]
└ @ Base loading.jl:1190
┌ Warning: Package FCA does not have Combinatorics in its dependencies:
│ - If you have FCA checked out for development and have
│   added Combinatorics as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with FCA
└ Loading Combinatorics into FCA from project dependency, future warnings for FCA are suppressed.


## DCT and DST orthogonal matrix

In [2]:
function dctmtx(n)
    cc, rr = [j for i = 0: n - 1, j = 0: n-1], [i for i = 0: n - 1, j = 0: n-1]
    c = sqrt(2 / n) * cos.(pi * (2*cc .+ 1) .* rr / (2 * n));
    c[1,:] = c[1,:] / sqrt(2);
    return c
end



dctmtx (generic function with 1 method)

In [3]:
function dstmtx(n)
    cc, rr = [j for i = 0: n - 1, j = 0: n-1], [i for i = 0: n - 1, j = 0: n-1]
    c = sqrt(2 / n) * sin.(pi * (2*cc .+ 1) .* (rr .+ 1) / (2 * n));
    c[n,:] = c[n,:] / sqrt(2);
    return c
end

dstmtx (generic function with 1 method)

## Image denoise

In [4]:
function trial(N, lambda)
    # generate image 
    M = Int(ceil(N/lambda));
    U,V = dstmtx(N), dctmtx(M);
    D = Diagonal(map(x -> (x - 1)^4, range(1/2/N, length=N, stop=1-1/2/N)))
    D = hcat(D, zeros(N, M - N))
    image = U*D*V';
    image = free_whiten([image]; mat = "rec")[1][1];
    
    # generate noise
    noise = randn(N,M);
    noise = free_whiten([noise]; mat = "rec")[1][1];
    
    X = [image, noise];
    Q = [1 1; -1 1]/sqrt(2);
    
    Y = Q*X;
    
    # use kurtosis-based FCA
    Qinv_kurt, _ = freecf(Y; mat = "rec", obj = "kurt", opt = "orth");
    err_kurt = err_pd(pinv(Qinv_kurt)*Q)
    
    # use entropy-based FCA
    Qinv_ent, _ = freecf(Y; mat = "rec", obj = "ent", opt = "orth");
    err_ent = err_pd(pinv(Qinv_ent)*Q)
    
    # use kurtosis-based ICA
    y = [vec(Yi) for Yi in Y]
    Qinv_ica, _ = icf(y; opt = "orth")
    err_ica = err_pd(pinv(Qinv_ica)*Q)
    
    return err_kurt, err_ent, err_ica
end

trial (generic function with 1 method)

In [ ]:
trialsize = 200;
Nlist = [Int64(ceil(2^pow)) for pow in collect(range(7, length = 5, stop = 9))]
lambda = 4/5;
err_kurt_record = zeros(size(Nlist, 1), trialsize);
err_ent_record = zeros(size(Nlist, 1), trialsize);
err_ica_record = zeros(size(Nlist, 1), trialsize);

for trial_idx in 1: trialsize
    @show trial_idx
    for N_idx in 1:length(Nlist)
        N = Nlist[N_idx]
        err_kurt, err_ent, err_ica = trial(N, lambda)
        err_kurt_record[N_idx, trial_idx] = err_kurt
        err_ent_record[N_idx, trial_idx] = err_ent
        err_ica_record[N_idx, trial_idx] = err_ica
    end
end

trial_idx = 1
trial_idx = 2
trial_idx = 3
trial_idx = 4


┌ Warning: Linesearch failed, using alpha = 0.0 and exiting optimization.
│ The linesearch exited with message:
│ Linesearch failed to converge, reached maximum iterations 50.
└ @ Optim C:\Users\Hao Wu\.julia\packages\Optim\Agd3B\src\utilities\perform_linesearch.jl:47
